In [4]:
import re
import requests as req

from bs4 import BeautifulSoup

In [ ]:
report_url = 'https://cfb.mn.gov/reports-and-data/viewers/campaign-finance/large-contribution-notices/'
report_r = req.get(report_url)
soup = BeautifulSoup(report_r.content)

In [ ]:
candidate_soup = soup.find('div',{'class':'candidate-reports col-md-6'})
pcf_soup = soup.find('div',{'class':'pcf-reports col-md-6'})
js_link_list = [dd.a['href'] for dd in candidate_soup.find_all('dd')]
print(js_link_list)

In [ ]:
def arg_dict2api(arg_dict):
    api_url = 'https://cfb.mn.gov/rptViewer/Main.php?do=viewPDF'
    _api_url = api_url
    for k, v in arg_dict.items():
        _api_url += f'&{k}={v}'
    return _api_url

def arg_jsl2api(js_link):
    js_re_pattern = "javascript:viewNoticePDF\((.+)\);"
    arg_keys = ['year','type','period','se','regnum','date']
    arg_tup = tuple(qarg.strip("'\"") for qarg in re.search(js_re_pattern,js_link).group(1).split(','))
    arg_dict =  {k:v for k, v in zip(arg_keys,arg_tup)}
    return arg_dict2api(arg_dict)

In [ ]:
api_url_list = [arg_jsl2api(js_link) for js_link in js_link_list]
print(api_url_list)

In [1]:
with open('report_pdf/24_notice_PrePrimary_0_19072_240726_114701.txt','r') as f:
    pdf_text = f.read()

In [2]:
pdf_text

'Large Contribution Notice\nCandidate / Committee Information\nCommittee: Jones, Katie District 61A Committee (19072)\nTreasurer: Nalezny, Brian\nPeriod: 07/23/2024 through 08/12/2024\nSubmission date: July 27, 2024\nReceived by the Board July 27, 2024\nContributor, Lender, or Endorser Information\nContribution received from\nName: Lawrence, Mary\nEmployer: Doctor\nAddress: 4415 E Lake Harriet Pkwy\nStreet:\nCity: Minneapolis State: MN Zip code: 55419\nDate: 07/26/2024 Amount: 1,000.00\nInkind: No Description:\nLoan: No\nCampaign Finance Reporter Online 1.0.4439 XSD Version: 2.6 Printed 07/27/2024\nPage 1\n\x0c'

In [41]:
grab_pattern = lambda p: re.search(p,pdf_text).groups()
committee = grab_pattern('Committee: (.+)')[0]
treasurer = re.search('Treasurer: (.+)',pdf_text).group(1)
period = re.search('Period: (.+)',pdf_text).group(1)
submitted = re.search('Submission date: (.+)',pdf_text).group(1)
received = re.search('Received by the Board (.+)',pdf_text).group(1)
name = re.search('Name: (.+)',pdf_text).group(1)
employer = grab_pattern('Employer: (.+)')
street = grab_pattern('Address: (.+)')
city, state, zip = re.search('City: (.+) State: (.+) Zip code: (.+)',pdf_text).groups()
date, amount = re.search('Date: (.+) Amount: (.+)',pdf_text).groups()
inkind, description = re.search('Inkind: (.+) Description:(.*)',pdf_text).groups() #description may be empty
loan = grab_pattern('Loan: (.+)')[0]

In [39]:
re.search('Committee: (.+)',pdf_text).groups()[0]

'Jones, Katie District 61A Committee (19072)'

('Minneapolis', 'MN', '55419')